# AnxLight Launcher v0.0.5 (v3 Architecture)
### Cell 1: Pre-Flight Setup
This cell clones/updates the AnxLight repository and runs the `scripts/pre_flight_setup.py` script. This script handles VENV creation, all dependency installations (Gradio, etc.), tunneling clients, and installation of all supported WebUIs. Run this cell first and wait for it to complete. This cell is designed to be run once, or infrequently when major updates to WebUIs or core dependencies are needed.

In [ ]:
# Cell 1: Pre-Flight Setup (v3)
import os
import subprocess
from pathlib import Path
import sys

print("--- AnxLight v3 Pre-Flight Setup Cell ---")

# --- Configuration ---
REPO_URL = "https://github.com/drf0rk/AnxLight.git"
REPO_DIR_NAME = "AnxLight"
BRANCH_NAME = "main"

# --- 1. Clone/Update Repository ---
print(f"--- Ensuring latest repository code from branch: {BRANCH_NAME} ---")
base_dir = Path("/content")
repo_path = base_dir / REPO_DIR_NAME

if not repo_path.exists():
    print(f"Cloning repository '{REPO_URL}' (branch: {BRANCH_NAME}) into '{repo_path}'...")
    subprocess.run(["git", "clone", "-b", BRANCH_NAME, REPO_URL, str(repo_path)], check=True)
else:
    print(f"Repository already exists at '{repo_path}'. Fetching updates from branch '{BRANCH_NAME}'...")
    os.chdir(str(repo_path))
    subprocess.run(["git", "fetch", "origin"], check=True)
    subprocess.run(["git", "checkout", BRANCH_NAME], check=True)
    subprocess.run(["git", "reset", "--hard", f"origin/{BRANCH_NAME}"], check=True)
    subprocess.run(["git", "pull", "origin", BRANCH_NAME], check=True)

os.chdir(str(repo_path))
print(f"Current directory: {os.getcwd()}")

# --- 2. Execute Pre-Flight Setup Script ---
print("\n--- Executing Pre-Flight Setup Script (scripts/pre_flight_setup.py) ---")
pre_flight_script_path = os.path.join(os.getcwd(), "scripts", "pre_flight_setup.py")

if os.path.exists(pre_flight_script_path):
    print(f"Running: {sys.executable} {pre_flight_script_path}")
    # Using Popen to stream output in real-time
    process = subprocess.Popen([sys.executable, pre_flight_script_path],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               text=True,
                               bufsize=1,
                               universal_newlines=True)
    for line in process.stdout:
        print(line, end='')
    process.wait()
    if process.returncode != 0:
        print(f"!!! ERROR: Pre-flight setup script failed with exit code {process.returncode}. Check output above. !!!")
    else:
        print("--- Pre-Flight Setup Script completed successfully. ---")
else:
    print(f"!!! ERROR: scripts/pre_flight_setup.py not found at {pre_flight_script_path}. Cannot proceed. !!!")

print("\n--- AnxLight Cell 1 (Pre-Flight Setup) Complete ---")
print("If no errors above, you can now run Cell 2 to launch the Gradio application.")

### Cell 2: Launch Gradio Application (v3)
Run this cell only after Cell 1 has completed successfully. This cell launches the AnxLight Gradio UI for configuring and starting your WebUI session.

In [ ]:
# Cell 2: Launch Gradio Application (v3)
import os
import sys
import subprocess
import traceback
from pathlib import Path

PROJECT_ROOT_STR = "/content/AnxLight"
PROJECT_ROOT = Path(PROJECT_ROOT_STR)
APP_SCRIPT_RELATIVE_PATH = "scripts/main_gradio_app.py"
VENV_NAME = "anxlight_venv" # Must match VENV_NAME in pre_flight_setup.py

print(f"--- AnxLight v3 Launch Cell ---")

# Ensure CWD is project root
if Path.cwd() != PROJECT_ROOT:
    print(f"Changing directory to {PROJECT_ROOT_STR}")
    os.chdir(PROJECT_ROOT_STR)

scripts_path_abs = PROJECT_ROOT / "scripts"
if str(scripts_path_abs) not in sys.path:
    sys.path.insert(0, str(scripts_path_abs))

try:
    from anxlight_version import ANXLIGHT_LAUNCHER_NOTEBOOK_VERSION
    print(f"--- AnxLight Launcher Notebook v{ANXLIGHT_LAUNCHER_NOTEBOOK_VERSION} (v3) ---")
except ImportError:
    print(f"--- AnxLight Launcher Notebook (v3 - version import failed) ---")

print(f"Current Project Root (cwd for main_gradio_app.py): {Path.cwd()}")

# --- Environment Setup for main_gradio_app.py ---
print("\n--- Setting up Environment for Gradio Application ---")
os.environ['IS_COLAB'] = 'true'
os.environ['PROJECT_ROOT'] = PROJECT_ROOT_STR
os.environ['home_path'] = PROJECT_ROOT_STR
os.environ['scr_path'] = PROJECT_ROOT_STR
os.environ['settings_path'] = str(PROJECT_ROOT / 'anxlight_config.json')
os.environ['venv_path'] = str(PROJECT_ROOT / VENV_NAME)

modules_dir_abs = PROJECT_ROOT / 'modules'
current_python_path = os.environ.get('PYTHONPATH', '')
new_python_path_parts = [str(modules_dir_abs), PROJECT_ROOT_STR, str(scripts_path_abs)]
if current_python_path:
    for part in current_python_path.split(os.pathsep):
        if part and part not in new_python_path_parts:
            new_python_path_parts.append(part)
os.environ['PYTHONPATH'] = os.pathsep.join(new_python_path_parts)

print(f"[Notebook Cell 2] ENV PROJECT_ROOT: {os.environ['PROJECT_ROOT']}")
print(f"[Notebook Cell 2] ENV home_path: {os.environ['home_path']}")
print(f"[Notebook Cell 2] ENV scr_path: {os.environ['scr_path']}")
print(f"[Notebook Cell 2] ENV settings_path: {os.environ['settings_path']}")
print(f"[Notebook Cell 2] ENV venv_path: {os.environ['venv_path']}")
print(f"[Notebook Cell 2] Updated PYTHONPATH: {os.environ['PYTHONPATH']}")

app_to_run_path_str = str(PROJECT_ROOT / APP_SCRIPT_RELATIVE_PATH)
python_in_venv_path = Path(os.environ['venv_path']) / ('Scripts' if sys.platform == 'win32' else 'bin') / 'python'

print(f"\n--- Launching Gradio Application --- ")
if python_in_venv_path.exists():
    print(f"Using VENV Python: {python_in_venv_path}")
    print(f"Executing: {python_in_venv_path} {app_to_run_path_str}")
    try:
        # Use Popen to allow Gradio to run and stream output without blocking notebook execution indefinitely
        # if Gradio's share=True is used. Otherwise, subprocess.run can be fine.
        process_env = os.environ.copy()
        # The Gradio app will print its own share URL if successful
        # Using Popen to make it non-blocking for the notebook cell if Gradio runs in background
        # For debugging, it might be better to see all output directly:
        gradio_process = subprocess.Popen([str(python_in_venv_path), app_to_run_path_str],
                                        env=process_env,
                                        cwd=PROJECT_ROOT_STR,
                                        stdout=subprocess.PIPE,
                                        stderr=subprocess.STDOUT,
                                        text=True,
                                        bufsize=1,
                                        universal_newlines=True)
        # Stream Gradio output
        for line in gradio_process.stdout:
            print(line, end='')
        gradio_process.wait()
        if gradio_process.returncode != 0:
             print(f"Warning: main_gradio_app.py exited with code {gradio_process.returncode}")

    except Exception as e:
        print(f"An error occurred during launch with VENV python: {type(e).__name__}: {e}")
        traceback.print_exc()
else:
    print(f"ERROR: VENV Python executable not found at {python_in_venv_path}. Pre-flight setup (Cell 1) might have failed.")
    print("Cannot launch main_gradio_app.py without its VENV Python.")

print(f"\n--- Gradio app launch sequence in notebook complete. ---")